Using the main.ipynb for training Transformers was too chaotic for me. This file will be ONLY contain code that is absolutely necessary for finding out how to train the Transformer model.

In [9]:
# if "preprocessing" folder in current folders -> cd back to original folder
%cd /content
import os
if os.path.exists("bsc-thesis"):
  # if bsc-thesis folder already exists; completely remove
  !rm -rf bsc-thesis

branch = "main"
!git clone --branch $branch https://github.com/maviddoerdijk/bsc-thesis.git
%cd bsc-thesis/src
%ls

/content
Cloning into 'bsc-thesis'...
remote: Enumerating objects: 211, done.
remote: Counting objects: 100% (211/211), done.
remote: Compressing objects: 100% (174/174), done.
remote: Total 211 (delta 114), reused 94 (delta 32), pack-reused 0 (from 0)
Receiving objects: 100% (211/211), 5.85 MiB | 7.78 MiB/s, done.
Resolving deltas: 100% (114/114), done.
/content/bsc-thesis/src
backtesting/  data/       main.py  preprocessing/
config/       main.ipynb  models/  utils/


In [10]:
!pip install ta
!pip install prophet
!pip install pykalman
!pip install PyWavelets

In [18]:
# Module imports
import pandas as pd
import numpy as np
from typing import Optional, Callable, Dict, Any
from sklearn.preprocessing import MinMaxScaler
from matplotlib import pyplot as plt

# Custom Imports
from models.statistical_models import create_dataset
from data.data_collection import gather_data
from data.scraper import load_cached_etf_tickers
from preprocessing.cointegration import find_cointegrated_pairs
from preprocessing.data_preprocessing import filter_pairs_data
from preprocessing.technical_indicators import combine_pairs_data
from models.statistical_models import default_normalize
from preprocessing.wavelet_denoising import wav_den
from preprocessing.filters import step_1_filter_remove_nans, step_2_filter_liquidity

In [19]:
startDateStr = '2008-10-01'
endDateStr = '2018-10-02' # documentation said that endDateStr is exclusive for both yahoofinance and the original code, but actually printing the shapes showed otherwise..
instrumentIdsNASDAQandNYSE = load_cached_etf_tickers()
data = gather_data(startDateStr, endDateStr, instrumentIdsNASDAQandNYSE)
data_close_filtered_1, data_open_filtered_1, data_high_filtered_1, data_low_filtered_1, data_vol_filtered_1, data_original_format_filtered_1 = step_1_filter_remove_nans(data['close'], data['open'], data['high'], data['low'], data['vol'], data)
data_close_filtered_2, data_open_filtered_2, data_high_filtered_2, data_low_filtered_2, data_vol_filtered_2, data_original_format_filtered_2 = step_2_filter_liquidity(data_close_filtered_1, data_open_filtered_1, data_high_filtered_1, data_low_filtered_1, data_vol_filtered_1, data_original_format_filtered_1)

scores, pvalues, pairs = find_cointegrated_pairs(data_original_format_filtered_2)
pairs_data = {key:value[1]  for (key, value) in pairs.items()}
pairs_data = sorted(pairs_data.items(), key=lambda x: x[1])
pairs_data_filtered = filter_pairs_data(pairs_data) # filter based on cointegration in such a way that we can simply pick the highest pair of stocks in the list.
# Extract the most highly cointegrated pairs
ticker_a, ticker_b = pairs_data_filtered[0][0][0], pairs_data_filtered[0][0][1]
pairs_timeseries_df = combine_pairs_data(data_close_filtered_2, data_open_filtered_2, data_high_filtered_2, data_low_filtered_2, data_vol_filtered_2, ticker_a, ticker_b)
# Note about pairs_timeseries_df: the timeseries output on which we should train are found in the key "Spread_Close"
# But, also the input features are the following keys: ['S1_rsi', 'S2_rsi', 'S1_mfi', 'S2_mfi', 'S1_adi', 'S2_adi', 'S1_vpt', 'S2_vpt', 'S1_atr', 'S2_atr', 'S1_bb_ma', 'S2_bb_ma', 'S1_adx', 'S2_adx', 'S1_ema', 'S2_ema', 'S1_macd', 'S2_macd', 'S1_dlr', 'S2_dlr']

[*********************100%***********************]  777 of 777 completed
ERROR:yfinance:
494 Failed downloads:
ERROR:yfinance:['USCL', 'FPXE', 'TEKX', 'GLCR', 'MYMG', 'AMZU', 'TSMG', 'IBBQ', 'AVS', 'NIKL', 'CAFG', 'OZEM', 'URNJ', 'LDSF', 'GBUG', 'VPLS', 'FMTM', 'QOWZ', 'QDTY', 'FMED', 'ESPO', 'SPYQ', 'IVEG', 'MSFU', 'EYEG', 'AAPB', 'SDTY', 'OOQB', 'BSVO', 'MCHS', 'BSMQ', 'EGGQ', 'STNC', 'XBIL', 'LGCF', 'MYCN', 'GPIX', 'FDIV', 'NERD', 'AMDD', 'VGUS', 'DWUS', 'USVN', 'MSFL', 'GLDY', 'CRWL', 'TSYY', 'PABD', 'BRTR', 'LEXI', 'CARY', 'BUFC', 'UFO', 'WRND', 'WTMU', 'MYMI', 'VFLO', 'IBGA', 'XYZG', 'CA', 'DUKH', 'EMXF', 'BSCU', 'LDEM', 'WABF', 'INDH', 'GTR', 'BSCW', 'DMXF', 'MCDS', 'TPLS', 'CTEC', 'DVAL', 'GPIQ', 'UYLD', 'BLCR', 'EBI', 'MVLL', 'CLOD', 'RGTX', 'IBGK', 'PCMM', 'OBIL', 'BTF', 'TSL', 'TDI', 'ERNZ', 'CORO', 'QSML', 'WNDY', 'COPJ', 'QQQS', 'PQJL', 'ROE', 'CEFA', 'SMCL', 'WCBR', 'CCNR', 'MYCL', 'BSJW', 'AVUQ', 'ELIL', 'KBAB', 'BSCY', 'MOOD', 'ARVR', 'QSIX', 'RAYS', 'BCLO', 'PSWD', 'MY

Completed 820 pairs


[('MBB', 'SHY'),
 ('IGIB', 'DVY'),
 ('IGIB', 'ONEQ'),
 ('IGIB', 'IFGL'),
 ('IGIB', 'BBH'),
 ('IGIB', 'PDP'),
 ('IGIB', 'ACWX'),
 ('IGIB', 'PEY'),
 ('IGIB', 'TUR'),
 ('IGIB', 'SOXX'),
 ('IGIB', 'TLT'),
 ('IGIB', 'PHO'),
 ('IGIB', 'IUSG'),
 ('IGIB', 'PRFZ'),
 ('IGIB', 'PID'),
 ('IGIB', 'IJT'),
 ('IGIB', 'ACWI'),
 ('IGIB', 'AAXJ'),
 ('IGIB', 'SHV'),
 ('IGIB', 'PFF'),
 ('IGIB', 'PKW'),
 ('IGIB', 'IEI'),
 ('IGIB', 'RTH'),
 ('IGIB', 'QTEC'),
 ('IGIB', 'IGF'),
 ('IGIB', 'FEX'),
 ('IGIB', 'PPH'),
 ('IGIB', 'SHY'),
 ('IGIB', 'QQQ'),
 ('IGIB', 'PIE'),
 ('IGIB', 'IBB'),
 ('IGIB', 'PNQI'),
 ('IGIB', 'SMH'),
 ('IGIB', 'EMB'),
 ('DVY', 'PRFZ'),
 ('DVY', 'PKW'),
 ('IFGL', 'USIG'),
 ('IFGL', 'PFF'),
 ('IFGL', 'IGSB'),
 ('IFGL', 'EMB'),
 ('USIG', 'ACWX'),
 ('USIG', 'IUSV'),
 ('USIG', 'PEY'),
 ('USIG', 'TUR'),
 ('USIG', 'SOXX'),
 ('USIG', 'TLT'),
 ('USIG', 'PHO'),
 ('USIG', 'PID'),
 ('USIG', 'ACWI'),
 ('USIG', 'AAXJ'),
 ('USIG', 'SHV'),
 ('USIG', 'PFF'),
 ('USIG', 'PKW'),
 ('USIG', 'IEI'),
 ('USIG', 'QT

In [34]:
# Set a bunch of variables based on the existing function `execute_kalman_workflow` (Note: Some are changed already)
pairs_timeseries: pd.DataFrame = pairs_timeseries_df
target_col: str = "Spread_Close"
burn_in: int = 30 # we remove the first 30 elements, because the largest window used for technical indicators is
train_frac: float = 0.90
dev_frac: float = 0.05   # remaining part is test
look_back: int = 1
denoise_fn: Optional[Callable[[pd.Series], np.ndarray]] = wav_den
scaler_factory: Callable[..., MinMaxScaler] = MinMaxScaler
scaler_kwargs: Optional[Dict[str, Any]] = {"feature_range": (0, 1)}
normalise_fn: Callable[[pd.Series], pd.Series] = default_normalize
delta: float = 1e-3
obs_cov_reg: float = 2.
trans_cov_avg: float = 0.01
obs_cov_avg: float = 1.
return_datasets: bool = False
verbose: bool = False

In [52]:
# def execute_transformer_workflow
if not target_col in pairs_timeseries.columns:
  raise KeyError(f"pairs_timeseries must contain {target_col}")

# burn the first 30 elements
pairs_timeseries_burned = pairs_timeseries.iloc[burn_in:].copy()

total_len = len(pairs_timeseries_burned)
train_size = int(total_len * train_frac)
dev_size   = int(total_len * dev_frac)
test_size  = total_len - train_size - dev_size # not used, but for clarity

train = pairs_timeseries_burned[:train_size]
dev   = pairs_timeseries_burned[train_size:train_size+dev_size] # aka validation
test  = pairs_timeseries_burned[train_size+dev_size:]

if verbose:
    print(f"Split sizes — train: {len(train)}, dev: {len(dev)}, test: {len(test)}")

if denoise_fn is not None: # denoise using wavelet denoising
    train = pd.DataFrame({col: denoise_fn(train[col]) for col in train.columns}) # TODO: unsure whether dev and test should also be denoised?

scaler = scaler_factory(**scaler_kwargs)
trainX_untr, trainX, trainY_untr, trainY = create_dataset(train.values, scaler=scaler, look_back=look_back) # note: len(train) = len(trainX_unr) + 1, this could cause problems
devX_untr,   devX,   devY_untr,   devY   = create_dataset(dev.values,  scaler=scaler, look_back=look_back)
testX_untr,  testX,  testY_untr,  testY  = create_dataset(test.values, scaler=scaler, look_back=look_back)
trainY_untr[:20]

# To-do's before going on
# TODO: Remove S1_close/open/high/low/volume, same for S2 (done in combine_pairs_data)
# TODO: add removing/adding these as features to the ablation study.


[array([27.81830352]),
 array([27.42025825]),
 array([25.9191175]),
 array([22.98625305]),
 array([20.5661885]),
 array([21.23151271]),
 array([24.11603916]),
 array([25.605551]),
 array([26.16966699]),
 array([26.46204422]),
 array([25.33065673]),
 array([25.72835342]),
 array([25.91998167]),
 array([25.70591191]),
 array([25.83366537]),
 array([26.33152235]),
 array([26.35160811]),
 array([26.2352556]),
 array([26.03820719]),
 array([25.75521362])]

Some examples of outputs for understanding the form of the data better:

`len(trainX_untr)`
```
2238
```

`len(trainX_untr[0])`
```
34
```

Context: The 34 features consist of
* 10 technical indicators for both S1 and S2 (total 20)
* S1_close/open/high/low/volume, same for S2 (total 10)
* Pair spreads: close, open, high, low (total 4)


`trainX_untr[0] `
```
array([ 2.76970068e+01,  4.91006247e+01,  2.89730484e+01,  4.91027293e+01,
        2.89891358e+01,  4.91343834e+01,  2.60513431e+01,  4.87784465e+01,
        5.03546207e+05,  7.43386097e+03,  4.49718063e+01,  5.82671806e+01,
        5.75577766e+01,  8.28358144e+01,  7.59406546e+01,  3.92425336e+02,
        1.05376719e+05, -9.14930577e+03,  1.56861725e+00,  7.63660812e-01,
        2.85638197e+01,  4.83799321e+01,  8.30115861e+00,  2.65305580e+01,
        2.84687992e+01,  4.85740999e+01,  2.27306259e-01,  4.38795633e-02,
       -1.31033890e+00,  1.00654755e-01, -6.89227038e+00, -5.61453974e+00,
       -5.72867758e+00, -8.18838280e+00])
```

`len(trainY_untr)`
```
2238
```


`len(trainY_untr)[0]`
```
1
```

`trainY_untr[0]`
```
array([27.81830352])
```

`trainY_untr[:20]`
```
[array([27.81830352]),
 array([27.42025825]),
 array([25.9191175]),
 array([22.98625305]),
 array([20.5661885]),
 array([21.23151271]),
 array([24.11603916]),
 array([25.605551]),
 array([26.16966699]),
 array([26.46204422]),
 array([25.33065673]),
 array([25.72835342]),
 array([25.91998167]),
 array([25.70591191]),
 array([25.83366537]),
 array([26.33152235]),
 array([26.35160811]),
 array([26.2352556]),
 array([26.03820719]),
 array([25.75521362])]
 ```